# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_csv = "output_data/cities.csv"

cities = pd.read_csv(cities_csv)
cities = cities.drop('Unnamed: 0',axis=1)

#For a smaller subset to try API calls. Comment before submit.
#cities = cities.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

In [4]:
locations = cities[["Latitude","Longitude"]]
weights = cities["Humidity"]

In [5]:
#Set up your map
fig = gmaps.figure(layout={
        'width': '800',
        'height': '800px',
        'padding': '1px',
        'border': '1px solid black',
        'center' : (0,0), 
        'zoom_level':'5'})
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

 A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

In [6]:
good_wdf = cities.loc[cities["Max Temp"] >= 70]
good_wdf = good_wdf.loc[cities["Max Temp"] <= 80]
good_wdf = good_wdf.loc[cities["Wind Speed"] <= 10]
good_wdf = good_wdf.loc[cities["Cloudiness"] == 0]

In [7]:
good_cities = good_wdf["City"].count()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
good_wdf["Hotel Name"] = "None"
good_wdf

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Hotel Name
31,Mossamedes,AO,71.11,79,0,3.27,-15.20,12.15,None
52,Etchoropo,MX,70.00,28,0,8.01,26.77,-109.67,None
54,Nioro,GM,75.15,62,0,3.47,13.35,-15.75,None
149,Jiwani,PK,73.58,62,0,6.93,25.05,61.74,None
155,Bara,NG,74.26,33,0,4.38,10.37,10.73,None
256,Nokaneng,BW,74.53,28,0,9.28,-19.67,22.27,None
272,Boyuibe,BO,73.96,47,0,6.22,-20.42,-63.28,None
276,Durban,ZA,71.60,94,0,4.70,-29.86,31.03,None
305,Dakar,SN,78.80,88,0,2.24,14.69,-17.44,None
324,Santa Isabel,BR,75.20,78,0,3.36,-23.32,-46.22,None


In [9]:
for x in range(good_cities):
    try:
        target_coordinates = f"{good_wdf.iloc[x,6]},{good_wdf.iloc[x,7 ]}"
        target_search = "hotel"
        target_radius = 5000
        target_type = "hotel"

        # set up a parameters dictionary
        params = {
            "location": target_coordinates,
            "keyword": target_search,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
        }

        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params)
        response = response.json()
        good_wdf.iloc[x,8] = response["results"][x]["name"]
        
    except:
        print(f"Hotel not found")
        good_wdf.iloc[x,8] = "No Hotel"

Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found


In [11]:
marker_locations = good_wdf[["Latitude","Longitude"]]

markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800'))

In [12]:
# NOTE: I changed this code.
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>

"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in good_wdf.iterrows()]
locations = good_wdf[["Latitude", "Longitude"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Hotel Chik Chik Namibe</dd>\n<dt>City</dt><dd>Mossamedes</dd>\n<dt>Country</dt><dd>AO</dd>\n</dl>\n\n',
 '\n<dl>\n<dt>Name</dt><dd>No Hotel</dd>\n<dt>City</dt><dd>Etchoropo</dd>\n<dt>Country</dt><dd>MX</dd>\n</dl>\n\n',
 '\n<dl>\n<dt>Name</dt><dd>No Hotel</dd>\n<dt>City</dt><dd>Nioro</dd>\n<dt>Country</dt><dd>GM</dd>\n</dl>\n\n',
 '\n<dl>\n<dt>Name</dt><dd>No Hotel</dd>\n<dt>City</dt><dd>Jiwani</dd>\n<dt>Country</dt><dd>PK</dd>\n</dl>\n\n',
 '\n<dl>\n<dt>Name</dt><dd>No Hotel</dd>\n<dt>City</dt><dd>Bara</dd>\n<dt>Country</dt><dd>NG</dd>\n</dl>\n\n',
 '\n<dl>\n<dt>Name</dt><dd>No Hotel</dd>\n<dt>City</dt><dd>Nokaneng</dd>\n<dt>Country</dt><dd>BW</dd>\n</dl>\n\n',
 '\n<dl>\n<dt>Name</dt><dd>No Hotel</dd>\n<dt>City</dt><dd>Boyuibe</dd>\n<dt>Country</dt><dd>BO</dd>\n</dl>\n\n',
 '\n<dl>\n<dt>Name</dt><dd>Blue Waters Hotel</dd>\n<dt>City</dt><dd>Durban</dd>\n<dt>Country</dt><dd>ZA</dd>\n</dl>\n\n',
 '\n<dl>\n<dt>Name</dt><dd>Le Djoloff</dd>\n<dt>City</dt><dd>Dakar

In [30]:
# Add marker layer ontop of heat map


# Display Map